In [1]:
import numpy as np
import TensorFrost as tf

tf.initialize(tf.opengl)

def modified_gram_schmidt(A):
    """
    Implements the Modified Gram-Schmidt orthogonalization to get the QR decomposition of matrix A.
    A = QR
    """
    A = A.astype(float)  # Ensure A is of float type
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    
    for i in range(n-1):
        R[i, i] = np.linalg.norm(A[:, i])
        Q[:, i] = A[:, i] / R[i, i]
        R[i, i+1:n] = np.dot(Q[:, i].T, A[:, i+1:n])
        A[:, i+1:n] -= np.outer(Q[:, i], R[i, i+1:n])
    R[n-1, n-1] = np.linalg.norm(A[:, n-1])
    Q[:, n-1] = A[:, n-1] / R[n-1, n-1]
    return Q, R

# Householder reflection
def householder_reflection(a):
    v = a.copy()
    v[0] = v[0] + np.copysign(np.linalg.norm(v), a[0])
    v = v / np.linalg.norm(v)
    return np.eye(len(a)) - 2 * np.outer(v, v)

# QR decomposition using Householder reflections
def qr_decomposition(A):
    m, n = A.shape
    Q = np.eye(m)
    R = A.copy()
    for i in range(min(m, n)):
        H = np.eye(m)
        H[i:, i:] = householder_reflection(R[i:, i])
        Q = Q @ H
        R = H @ R
    return Q, R

#dynamic size QR decomposition
def QRDecomposition():
    A = tf.input([-1, -1], tf.float32)

    m, n = A.shape
    Q = tf.zeros([m, n])
    R = tf.zeros([n, n])

    j = tf.index(0, [m])

    def loop_body(i):
        norm2 = tf.zeros([1], tf.float32)
        def loop_body1(it):
            norm2.set(norm2 + A[it, i] ** 2)
        tf.loop(loop_body1, 0, m, 1)
        R[i, i] = tf.sqrt(norm2)
        Q[j, i] = A[j, i] / R[i, i]
        
        t, = tf.index_grid([i+1], [n])
        dotprod = tf.zeros(t.shape, tf.float32)
        def loop_body2(it):
            dotprod.set(dotprod + Q[it, i] * A[it, t])
        tf.loop(loop_body2, 0, m, 1)
        R[i, t] = dotprod
        
        p, k = tf.index_grid([0, i+1], [m, n])
        A[p, k] -= Q[p, i] * R[i, k]

        #p, k = tf.index_grid([0, i+1], [m, n])
        #R[i, t] = (Q[p, i] * A[p, k]).sum(axis=0) #TODO: implement sum reduction

    tf.loop(loop_body, 0, n-1, 1)

    norm2 = tf.zeros([1], tf.float32)
    def loop_body1(it):
        norm2.set(norm2 + A[it, n-1] ** 2)
    tf.loop(loop_body1, 0, m, 1)
    R[n-1, n-1] = tf.sqrt(norm2)
    Q[j, n-1] = A[j, n-1] / R[n-1, n-1]

    return [Q, R]

qr = tf.compile(QRDecomposition)


TensorFrost module loaded!
QRDecomposition:
  Kernel count: 8
  Intermediate buffers: 0
  Host readbacks: 0
  Host writes: 0
  Lines of generated code: 386
  IR Compile time: 1.468200 ms
  Compiler time: 1338.116455 ms



In [2]:
#generate random matrix
A = np.random.rand(5, 5)

#compute QR decomposition
Q, R = modified_gram_schmidt(A)

#compute QR decomposition using TensorFrost
Atf = tf.tensor(A)
Qtf, Rtf = qr(Atf)
Qnp = Qtf.numpy
Rnp = Rtf.numpy

#check if QR decomposition is correct
print("QR decomposition is correct:", np.allclose(A, np.dot(Q, R)))
print("QR decomposition using TensorFrost is correct:", np.allclose(A, np.dot(Qnp, Rnp)))

#check error
print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qnp, Rnp)))

#print Q and R
print("Q:\n", Qnp)
print("R:\n", Rnp)


QR decomposition is correct: True
QR decomposition using TensorFrost is correct: True
Error: 2.4196749845665633e-16
Error using TensorFrost: 1.7463660958911132e-07
Q:
 [[ 0.12900747 -0.04633267  0.85354614  0.49868265  0.06312579]
 [ 0.10285251  0.70631176 -0.15278302  0.3730224  -0.57276213]
 [ 0.8699651  -0.30220303 -0.01054917 -0.19219196 -0.3387996 ]
 [ 0.4263036   0.1968207  -0.35251665  0.42409173  0.6894963 ]
 [ 0.18494472  0.6073811   0.35176292 -0.6287911   0.2788697 ]]
R:
 [[0.7926238  1.3643625  1.2382593  0.69185907 0.9429792 ]
 [0.         1.165894   0.6241264  0.31904197 0.7199794 ]
 [0.         0.         0.90213925 0.46822688 0.49117544]
 [0.         0.         0.         0.2116255  0.7114116 ]
 [0.         0.         0.         0.         0.4332226 ]]


In [3]:
#performance test
import time
A = np.random.rand(5000, 5000).astype(np.float32)

#naive NumPy QR decomposition
#start = time.time()
#Q, R = modified_gram_schmidt(A)
#print("Time for naive NumPy QR decomposition:", time.time() - start)

#TensorFrost QR decomposition
Atf = tf.tensor(A)
start = time.time()
Qtf, Rtf = qr(Atf)
print("Time for TensorFrost QR decomposition:", time.time() - start)

#householder QR decomposition
#start = time.time()
#Q, R = qr_decomposition(A)
#print("Time for householder QR decomposition:", time.time() - start)

#built-in NumPy QR decomposition
start = time.time()
Q, R = np.linalg.qr(A)
print("Time for built-in NumPy QR decomposition:", time.time() - start)

print("Error:", np.linalg.norm(A - np.dot(Q, R)))
print("Error using TensorFrost:", np.linalg.norm(A - np.dot(Qtf.numpy, Rtf.numpy)))

Time for TensorFrost QR decomposition: 0.1770005226135254
Time for built-in NumPy QR decomposition: 3.8822364807128906
Error: 0.0012288705
Error using TensorFrost: 0.0018782484
